# Sampleassembly for single crystal silicon
* a = 5.431
* u = -1,1,-1 / sqrt(3), v = 2,1,-1 / sqrt(6)
* T=100K phonon data

## Step 0. some goodies

In [1]:
%matplotlib notebook
from matplotlib import pyplot as plt
import numpy as np
import histogram.hdf as hh, histogram as H

/home/lj7/anaconda2/envs/mcvine/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
workdir = '/SNS/users/lj7/simulations/samples/Si/demo'
!mkdir -p {workdir}
%cd "{workdir}"

/SNS/users/lj7/simulations/samples/Si/demo


In [17]:
!rsync -avzL /SNS/users/lj7/simulations/ARCS/Si/April2016/scan-T100K/sampleassembly/ {workdir}/

sending incremental file list
./
Si-scatterer.xml
sampleassembly.xml
Si-phonons/
Si-phonons/DOS
Si-phonons/FORCE_CONSTANTS
Si-phonons/Omega2
Si-phonons/POSCAR
Si-phonons/Polarizations
Si-phonons/Qgridinfo
Si-phonons/compute.py
Si-phonons/dos-withbroadening.dat
Si-phonons/run.sh

sent 72,299,998 bytes  received 240 bytes  9,640,031.73 bytes/sec
total size is 91,480,768  speedup is 1.27


In [6]:
%%file sampleassembly.xml
<?xml version="1.0"?>

<!DOCTYPE SampleAssembly>

<SampleAssembly name="Si"
   max_multiplescattering_loops_among_scatterers="1"
   max_multiplescattering_loops_interactM_path1="4"
   min_neutron_probability=".1"
 >
  
  <PowderSample name="Si" type="sample">
    <Shape>
      <hollowCylinder in_radius="5./8*inch" out_radius="1.*inch" height="1.5*inch"/>
    </Shape>
    <Phase type="crystal">
      <ChemicalFormula>Si</ChemicalFormula>
      <xyzfile>Si.xyz</xyzfile>
    </Phase>
  </PowderSample>
  
  <LocalGeometer registry-coordinate-system="InstrumentScientist">
    <Register name="Si" position="(0,0,0)" orientation="(0, 0, 0)"/>
  </LocalGeometer>

  <Environment temperature="300*K"/>

</SampleAssembly>

Overwriting sampleassembly.xml


In [25]:
!cat {workdir}/Si.xyz

2
0.0	2.71526503565	2.71526503565 	2.71526503565	0.0	2.71526503565 	2.71526503565	2.71526503565	0.0
Si	-0.125 -0.125 -0.125
Si 	0.125  0.125  0.125



In [7]:
a,b,c=5.431,5.431,5.431

from numpy import sin, cos, deg2rad
a1 = [a, 0, 0]
a2 = [0, b, 0]
a3 = [0, 0, c]

twopi = 2*np.pi
f = twopi/np.dot(a1, np.cross(a2, a3))
b1 = np.cross(a2,a3) * f
b2 = np.cross(a3,a1) * f
b3 = np.cross(a1,a2) * f
b1,b2,b3

(array([ 1.15691131,  0.        ,  0.        ]),
 array([ 0.        ,  1.15691131,  0.        ]),
 array([ 0.        ,  0.        ,  1.15691131]))

#### Orientation of the kernel
Here we need to use the convention that z is along beam, y vertical up.
The kernel has its own cartesian axis e’x e’y e’z. Because we know
u = -11-1 v = 21-1
, we have the basis vectors of the instrument coordinate system

    ez along -b1 + b2 -b3
    ex along 2b1+b2-b3
    

So we have them in the reciprocal cartesian axis

In [8]:
ez = -b1 + b2 - b3; ez /= np.linalg.norm(ez)
ex1 = 2*b1 + b2 - b3; print ex1
ey = np.cross(ez, ex1); ey/=np.linalg.norm(ey)
ex = np.cross(ey, ez)
ex, ey, ez

[ 2.31382261  1.15691131 -1.15691131]


(array([ 0.81649658,  0.40824829, -0.40824829]),
 array([ 0.        , -0.70710678, -0.70710678]),
 array([-0.57735027,  0.57735027, -0.57735027]))

The rotation matrix is define as
```
    ex.ex’ ex.ey’ ex.ez’ 
R = ey.ex’ ey.ey’ ey.ez’
    ez.ex’ ez.ey’ ez.ez’
```
So

In [9]:
R = np.array([ ex, ey, ez ])
Rflat = R.copy(); Rflat.shape = -1,
R, Rflat

(array([[ 0.81649658,  0.40824829, -0.40824829],
        [ 0.        , -0.70710678, -0.70710678],
        [-0.57735027,  0.57735027, -0.57735027]]),
 array([ 0.81649658,  0.40824829, -0.40824829,  0.        , -0.70710678,
        -0.70710678, -0.57735027,  0.57735027, -0.57735027]))

Now we add an extra kernel parameter:

In [10]:
'orientation="%s"' % (','.join(str(e) for e in Rflat),)

'orientation="0.816496580928,0.408248290464,-0.408248290464,0.0,-0.707106781187,-0.707106781187,-0.57735026919,0.57735026919,-0.57735026919"'

In [ ]:
# modify Si-scatterer.xml to specify the orientation

In [11]:
!cat {workdir}/Si-scatterer.xml

<?xml version="1.0"?>

<!DOCTYPE scatterer>

<!-- mcweights: monte-carlo weights for 3 possible processes:
     absorption, scattering, transmission -->
<homogeneous_scatterer 
  mcweights="0, 9, 1"
  max_multiplescattering_loops="4"
  >

    <!-- coherent -->
    <KernelContainer average="yes">

      <Phonon_CoherentInelastic_SingleXtal_Kernel weight="1.">
	<LinearlyInterpolatedDispersion idf-data-path="Si-phonons"/>
      </Phonon_CoherentInelastic_SingleXtal_Kernel>

      <MultiPhonon_Kernel Qmax="14/angstrom" dQ="0.05/angstrom" Emax="80*meV" weight="1.">
        <LinearlyInterpolatedDOS ascii-path="Si-phonons/dos-withbroadening.dat"/>
      </MultiPhonon_Kernel>
      
    </KernelContainer>

</homogeneous_scatterer>


In [20]:
%%file {workdir}/Si-scatterer.xml
<?xml version="1.0"?>

<!DOCTYPE scatterer>

<!-- mcweights: monte-carlo weights for 3 possible processes:
     absorption, scattering, transmission -->
<homogeneous_scatterer 
  mcweights="0, 7, 3"
  max_multiplescattering_loops="4"
  >

    <!-- coherent -->
    <KernelContainer average="yes">

      <Phonon_CoherentInelastic_SingleXtal_Kernel 
        weight="1."
        orientation="0.816496580928,0.408248290464,-0.408248290464,0.0,-0.707106781187,-0.707106781187,-0.57735026919,0.57735026919,-0.57735026919">
	<LinearlyInterpolatedDispersion idf-data-path="Si-phonons"/>
      </Phonon_CoherentInelastic_SingleXtal_Kernel>

      <MultiPhonon_Kernel Qmax="14/angstrom" dQ="0.05/angstrom" Emax="80*meV" weight="1.">
        <LinearlyInterpolatedDOS ascii-path="Si-phonons/dos-withbroadening.dat"/>
      </MultiPhonon_Kernel>
      
    </KernelContainer>

</homogeneous_scatterer>

Overwriting /SNS/users/lj7/simulations/samples/Si/demo/Si-scatterer.xml


In [21]:
!cat {workdir}/Si-scatterer.xml

<?xml version="1.0"?>

<!DOCTYPE scatterer>

<!-- mcweights: monte-carlo weights for 3 possible processes:
     absorption, scattering, transmission -->
<homogeneous_scatterer 
  mcweights="0, 9, 1"
  max_multiplescattering_loops="4"
  >

    <!-- coherent -->
    <KernelContainer average="yes">

      <Phonon_CoherentInelastic_SingleXtal_Kernel 
        weight="1."
        orientation="0.816496580928,0.408248290464,-0.408248290464,0.0,-0.707106781187,-0.707106781187,-0.57735026919,0.57735026919,-0.57735026919">
	<LinearlyInterpolatedDispersion idf-data-path="Si-phonons"/>
      </Phonon_CoherentInelastic_SingleXtal_Kernel>

      <MultiPhonon_Kernel Qmax="14/angstrom" dQ="0.05/angstrom" Emax="80*meV" weight="1.">
        <LinearlyInterpolatedDOS ascii-path="Si-phonons/dos-withbroadening.dat"/>
      </MultiPhonon_Kernel>
      
    </KernelContainer>

</homogeneous_scatterer>

In [22]:
!rm {workdir}/*~

In [24]:
!ls -l {workdir}

total 16
-rw-r--r-- 1 lj7 users  730 Apr 25 13:48 sampleassembly.xml
drwxr-xr-x 1 lj7 users 4096 Apr 25 13:48 Si-phonons
-rw-r--r-- 1 lj7 users  889 Jun 22 10:20 Si-scatterer.xml
-rw-r--r-- 1 lj7 users  149 Apr 25 13:48 Si.xyz
